# Imports

In [65]:
import random
import csv
import os
import pandas as pd
import numpy as np
from scipy.stats import norm, skewnorm


In [66]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

# Parameters

In [67]:
AIRLINE_COUNT=3
DAILY_FLIGHT_COUNT=15
DAILY_FLIGHT_OPTIONS = 30
ORIGIN_COUNT=10
AIRCRAFT_TYPES = ['Boeing 737', 'Airbus A320', 'Boeing 777', 'Airbus A330', 'Boeing 787', 'Embraer E175']
HOLIDAY_COUNT = 20
YEAR_FROM=2010
YEAR_TO=2022
SEED = 41
AIRCRAFT_WEIGHTS_DELAY = {'Boeing 737':15, 'Airbus A320':10, 'Boeing 777':5, 'Airbus A330':-5, 'Boeing 787':-5, 'Embraer E175':-10}
AIRCRAFT_WEIGHTS_VAR_DELAY = {'Boeing 737':5, 'Airbus A320':4, 'Boeing 777':2, 'Airbus A330':1, 'Boeing 787':1, 'Embraer E175':4}

# Generate

In [68]:
# set seed for reproducibility
#random.seed(SEED)

def mixture_rvs(size, w1, mu1, var1, mu2, var2):
    w2 = 1 - w1
    # Draw uniform random numbers to decide which distribution to sample from
    mixture_component = np.random.choice([0, 1], p=[w1, w2], size=size)

    # Sample from the appropriate normal distribution for each component
    samples = np.where(
        mixture_component == 0,
        norm.rvs(loc=mu1, scale=np.sqrt(var1), size=size),
        norm.rvs(loc=mu2, scale=np.sqrt(var2), size=size)
    )

    return samples

class Airline:
    def __init__(self, code):
        self.code = code
        self.seeds = [random.random() for i in range(6)]
        # delay center between -30 and 30
        self.delay_center = random.randint(-20, 35)
        self.delay_var = random.randint(0,40)

    def __str__(self):
        return self.code

    def __repr__(self):
        return self.code

def generate_origin(current):
    while True:
        # creates a random tree letter origin code
        origin = []
        origin.append(chr(random.randint(65, 90)))
        origin.append(chr(random.randint(65, 90)))
        origin.append(chr(random.randint(65, 90)))
        origin = ''.join(origin)
        # checks if the origin code is already in use
        if origin not in current:
            return origin

def generate_flight(airlines, k, origins):
    flight = []

    # flight k
    flight.append(k)

    # flight id (LL3050)
    airline = random.choices(airlines, weights=[7,2,1])[0]
    flight.append(airline)

    # weighted for aircraft type
    aircraft_type = random.choices(AIRCRAFT_TYPES, weights=[
            10 * airline.seeds[0],
            10 * airline.seeds[1],
            6 * airline.seeds[2],
            5 * airline.seeds[3],
            5 * airline.seeds[4],
            4 * airline.seeds[5]])[0]
    flight.append(aircraft_type)

    # 80% is schengen
    flight.append(random.choices(['schengen', 'non-schengen'], weights=[8, 2])[0])

    # random origin
    flight.append(random.choices(origins, weights=[
            random.randint(1,10) * airline.seeds[0],
            random.randint(1,10) * airline.seeds[1],
            random.randint(1,10) * airline.seeds[2],
            random.randint(1,10) * airline.seeds[3],
            random.randint(1,10) * airline.seeds[4],
            random.randint(1,10) * airline.seeds[5],
            random.randint(1,10) * airline.seeds[2],
            random.randint(1,10) * airline.seeds[3],
            random.randint(1,10) * airline.seeds[4],
            random.randint(1,10) * airline.seeds[5]])[0])

    # arrival time
    # using a normal distribution centered at 9:00 and another centered at 17 with a standard deviation of 2 hours
    # create a normal centered on 9, var 2
    arrival_time = mixture_rvs(1, 0.5, 9, 2, 17, 2)[0]
    flight.append(arrival_time)

    # staying time is normal mu=3, var=1, minimum should be 1h
    staying_time = max(2, int(norm.rvs(loc=3.5, scale=1, size=1)[0]))
    departure_time = arrival_time + staying_time
    flight.append(departure_time)

    return flight

def generate_airline(current):
    while True:
        # creates a random two letter airline code
        airline = []
        airline.append(chr(random.randint(65, 90)))
        airline.append(chr(random.randint(65, 90)))
        airline = ''.join(airline)
        # checks if the airline code is already in use
        if airline not in current:
            return Airline(airline)

def generate_holiday(current):
    while True:
        # picks a random number between 1 and 365
        holiday = random.randint(1, 365)
        # checks if the holiday is already in use
        if holiday not in current:
            return holiday

def generate_real_flight(base_flight, year, day, holidays, origins_weights, origins_weights_var, noise_list):
    flight = []
    flight.extend(base_flight)

    delay_center = base_flight[1].delay_center
    delay_var = base_flight[1].delay_var
    # delay is normal on the center with 15 var
    delay = skewnorm.rvs(a = 6, loc=delay_center, scale=delay_var, size=1)[0]

    # in holiday delay center is worse in plus between 10 and 60
    is_holiday = day in holidays
    if is_holiday:
        # use a normal
        delay += skewnorm.rvs(a = 4, loc=25, scale=8, size=1)[0]

    # in weekend delay center is worse in plus between 10 and 30
    is_weekend = day % 7 == 0 or day % 7 == 6
    if is_weekend:
        # use a normal
        delay += skewnorm.rvs(a = 4, loc=15, scale=10, size=1)[0]

    delay += norm.rvs(loc=AIRCRAFT_WEIGHTS_DELAY[base_flight[2]], scale=AIRCRAFT_WEIGHTS_VAR_DELAY[base_flight[2]], size=1)[0]

    delay += norm.rvs(loc=origins_weights[base_flight[4]], scale=origins_weights_var[base_flight[4]], size=1)[0]

    # # in some airlines delay center is worse in plus between 10 and 30
    # is_worse_airline = base_flight[1].code in worse_airlines
    # if is_worse_airline:
    #     # use a normal
    #     delay += norm.rvs(loc=20, scale=5, size=1)[0]

    r = random.random()
    if r> 0.995:
        noise = random.random() * 60
    elif r > 0.9:
        noise = random.random() * 30
    elif r > 0.8:
        noise = random.random() * 10
    elif r > 0.6:
        noise = random.random() * 5
    else:
        noise = random.random() - 0.5

    #save the noise on a list
    noise_list.append(noise)

            
    delay += noise

    flight.append(day)
    flight.append(year)
    flight.append(is_holiday)
    flight.append(delay)

    return flight

#funcition check the last file name and create a new one
def check_file_name():
    files = os.listdir('../data/data_study')
    files = [file for file in files if file.startswith('flights_with_noise')]
    if len(files) == 0:
        return 'flights_with_noise_1.csv'
    else:
        files.sort()
        last_file = files[-1]
        number = int(last_file.split('_')[3].split('.')[0])
        return f'flights_with_noise_{number+1}.csv'

#function thats saves flights on a csv file
def save_flights(flights, noise_list):

    file_name = check_file_name()

    with open(f'../data/data_study/{file_name}', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["flight_id", "airline", "aircraft_type", "schengen", "origin", "arrival_time", "departure_time", "day", "year", "is_holiday", "delay"])
        for flight in flights:
            writer.writerow(flight)

    #save the noise on a csv file name similiar as the flights file
    with open(f'../data/data_study/noise_{file_name}', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["noise"])
        for noise in noise_list:
            writer.writerow([noise])


def generate_dataset():
    noise_list = []
    airlines = []

    for i in range(AIRLINE_COUNT):
        airlines.append(generate_airline(airlines))

    # airlines with worse delays
    # worse_airlines = [airline.code for airline in random.choices(airlines, k=5)]

    origins = []
    for i in range(ORIGIN_COUNT):
        origins.append(generate_origin(origins))

    origins_weights = {origem:random.randint(-15,15) for origem in origins}
    origins_weights_var = {origem:random.randint(0,6) for origem in origins}

    #dentro da função generate_holiday, verificar se o if já não garante a condição de não repetição então não precisa ser set pode ser list
    holidays = set()
    for i in range(HOLIDAY_COUNT):
        h = generate_holiday(holidays)
        holidays.add(h)

    daily_flights_options = []
    for i in range(1, DAILY_FLIGHT_OPTIONS + 1):
        daily_flights_options.append(generate_flight(airlines, i, origins))

    flights = []
    for year in range(YEAR_FROM, YEAR_TO+1):
        # TODO voos de dias especificos
        # voos de natal
        for day in range(365):
            daily_flights = random.choices(daily_flights_options, k = DAILY_FLIGHT_COUNT)
            for base_flight in daily_flights:
                flights.append(generate_real_flight(base_flight, year, day, holidays, origins_weights, origins_weights_var, noise_list))
    
    save_flights(flights, noise_list)

# Analysis

In [69]:
#read all csv files and save on a list of Pandas DataFrames
def read_csv_files():
    files = os.listdir('../data/data_study')
    files = [file for file in files if file.startswith('flights')]
    dfs = []
    for file in files:
        dfs.append(pd.read_csv(f'../data/data_study/{file}'))
    return dfs


In [70]:
# generate_dataset()

In [71]:
dfs = read_csv_files()

#print the mean of delay of each file
for i in range(len(dfs)):
    print(f"mean of delay of file {i+1}: {dfs[i]['delay'].mean()}")

mean of delay of file 1: 12.548378015698628
mean of delay of file 2: 17.071611010697268
mean of delay of file 3: 62.470021028987695
mean of delay of file 4: 54.301755444776866
mean of delay of file 5: 66.55132129549682
mean of delay of file 6: 23.28533704475026
mean of delay of file 7: 48.99147281973866
mean of delay of file 8: 43.36273800655481
mean of delay of file 9: 28.295264361113382


In [72]:
#read the noise csv file and save on a list of Pandas DataFrames
def read_noise_files():
    files = os.listdir('../data/data_study')
    files = [file for file in files if file.startswith('noise')]
    dfs = []
    for file in files:
        dfs.append(pd.read_csv(f'../data/data_study/{file}'))
    return dfs

noise_dfs = read_noise_files()

#print the mean noise of each file
for i in range(len(noise_dfs)):
    print(f"mean of noise of file {i}: {noise_dfs[i]['noise'].mean()}")

mean of noise of file 0: 2.555992282326937
mean of noise of file 1: 2.589714848528952
mean of noise of file 2: 2.614312254509292
mean of noise of file 3: 2.5805435377098584
mean of noise of file 4: 2.5886580713841645
mean of noise of file 5: 2.571388204108151
mean of noise of file 6: 2.588789049362976


# Machine learing

In [73]:
#read the only metric csv file and save on a DataFrames if exists else create a empty DataFrame
def read_metrics_file():
    files = os.listdir('../data/')
    files = [file for file in files if file.startswith('metrics')]
    if len(files) == 0:
        return pd.DataFrame(columns=['file'])
    else:
        return pd.read_csv(f'../data/{files[0]}')

In [74]:
df_metrics = read_metrics_file()

for ind, dados in enumerate(dfs):

    #only run if the file is not on the metrics file
    if ind not in df_metrics['file'].values:

        categorical_vars = ['airline', 'aircraft_type', 'origin']

        # Perform one-hot encoding
        df_encoded_4 = pd.get_dummies(dados, columns=categorical_vars, dtype=int)

        df_encoded_4['is_holiday'] = df_encoded_4['is_holiday'].map({False: 0, True: 1})
        df_encoded_4['schengen'] = df_encoded_4['schengen'].map({'non-schengen': 0, 'schengen': 1})
        df_encoded_4['is_weekend'] = df_encoded_4['day'].apply(lambda day: day % 7 == 0 or day % 7 == 6)

        X = df_encoded_4.drop(['flight_id', 'day', 'year','departure_time', 'delay'], axis=1)
        y = df_encoded_4['delay']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # random_state=42

        model = RandomForestRegressor()
        rfe = RFE(model)
        pipeline = Pipeline(steps=[('s',rfe),('m',model)])
        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)

        mse = mean_squared_error(y_test, y_pred)

        rmse = mean_squared_error(y_test, y_pred, squared=False)

        r2 = r2_score(y_test, y_pred)

        #concat the metrics mae, mse, rmse, s2 of each file on a dataframe and the noise mean
        # the first put none on noise column because the first file doesn't have noise
        try:
            noise = noise_dfs[ind-1]['noise'].mean()
        except:
            noise = None
        
        df_metrics = pd.concat([df_metrics, pd.DataFrame({'file': ind, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2, 'noise_mean': noise}, index=[ind])])

#add the delay metrics, max, min, mean, mediam and std on the dataframe
df_metrics['delay_median'] = df_metrics['file'].apply(lambda file: dfs[file]['delay'].median())
df_metrics['delay_mean'] = df_metrics['file'].apply(lambda file: dfs[file]['delay'].mean())
df_metrics['delay_std'] = df_metrics['file'].apply(lambda file: dfs[file]['delay'].std())
df_metrics['delay_min'] = df_metrics['file'].apply(lambda file: dfs[file]['delay'].min())
df_metrics['delay_max'] = df_metrics['file'].apply(lambda file: dfs[file]['delay'].max())

#save the metrics on a csv file
df_metrics.to_csv('../data/metrics.csv', index=False)

# Resultados

In [83]:
#read metrics csv file and save on a Pandas DataFrame order by r2
df_metrics = pd.read_csv('../data/metrics.csv')
df_metrics = df_metrics.sort_values(by=['r2'], ascending=False)
df_metrics
# df_metrics[['file', 'r2', 'noise_mean', 'delay_median', 'delay_mean']]

,file,mae,mse,rmse,r2,noise_mean,delay_median,delay_mean,delay_std,delay_min,delay_max
6,6,7.935302,108.246762,10.404170,0.880453,2.571388,47.767941,48.991473,24.054547,-34.903859,225.296308
7,7,7.935309,108.258770,10.404747,0.880439,NaN,45.772060,43.362738,30.139726,-38.267560,170.844093
4,4,12.093037,240.966131,15.523084,0.770999,2.580544,64.289454,66.551321,27.215951,-15.112264,216.795404
8,8,10.095656,200.523411,14.160629,0.760132,NaN,24.453132,28.295264,29.200587,-41.130285,183.495870
0,0,10.067814,185.717082,13.627805,0.718328,NaN,9.740454,12.548378,23.125349,-41.028033,125.632352
2,2,16.874026,512.042847,22.628364,0.678183,2.589715,62.573630,62.470021,36.396846,-40.739357,249.020577
5,5,10.481181,203.082409,14.250699,0.648650,2.588658,17.844718,23.285337,32.442194,-46.438830,199.753918
1,1,17.258896,489.252073,22.119043,0.624955,2.555992,14.331179,17.071611,25.559476,-41.972611,208.194267
3,3,17.268755,469.572288,21.669617,0.366505,2.614312,54.461115,54.301755,39.984443,-33.120119,240.138531
